In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('../data/data.csv')
# убюраем ненужные нам фичи из датасета
data.drop('campaign_clicks', inplace=True, axis=1)
# уже знаем из 1 дз, что не нужно
data.drop('impressions', inplace=True, axis=1)
data.head()

,date_time,zone_id,banner_id,oaid_hash,os_id,country_id,banner_id0,rate0,g0,coeff_sum0,banner_id1,rate1,g1,coeff_sum1,clicks
0,2021-09-27 00:01:30.000000,0,0,5664530014561852622,0,0,1240,0.067,0.035016,-7.268846,0,0.010,0.049516,-5.369901,1
1,2021-09-26 22:54:49.000000,1,1,5186611064559013950,0,1,1,0.002,0.054298,-2.657477,269,0.004,0.031942,-4.449220,1
2,2021-09-26 23:57:20.000000,2,2,2215519569292448030,0,0,2,0.014,0.014096,-3.824875,21,0.014,0.014906,-3.939309,1
3,2021-09-27 00:04:30.000000,3,3,6262169206735077204,1,1,3,0.012,0.015232,-3.461357,99,0.006,0.050671,-3.418403,1
4,2021-09-27 00:06:21.000000,4,4,4778985830203613115,1,0,4,0.019,0.051265,-4.009026,11464230,6.790,0.032005,-2.828797,1


In [3]:
# Заметим, что в coeff_sum0 есть отрицательные числа. Посмотрим, где есть ещё
data.min()

date_time     2021-09-01 00:02:49.000000
zone_id                                0
banner_id                              0
oaid_hash                  1116910879938
os_id                                  0
country_id                             0
banner_id0                             0
rate0                                0.0
g0                             -0.017637
coeff_sum0                      -8.58897
banner_id1                             0
rate1                                0.0
g1                             -0.069839
coeff_sum1                     -9.562188
clicks                                 0
dtype: object

In [4]:
# go и g1 по условию являются стандартными отклонениями. Удалим те, что имеют отрицательные значения.
data = data[data['g0'] >= 0]
data = data[data['g1'] >= 0]
data.min()

date_time     2021-09-01 00:02:49.000000
zone_id                                0
banner_id                              0
oaid_hash                  1116910879938
os_id                                  0
country_id                             0
banner_id0                             0
rate0                                0.0
g0                                   0.0
coeff_sum0                     -8.419295
banner_id1                             0
rate1                                0.0
g1                                   0.0
coeff_sum1                     -9.562188
clicks                                 0
dtype: object

In [5]:
# добавим ещё одну фичу с часами
data['hour'] = pd.to_datetime(data['date_time']).dt.hour
data['date'] = pd.to_datetime(data['date_time']).dt.date

In [6]:
# Находим дату последнего дня
data['date'].max()

datetime.date(2021, 10, 2)

In [7]:
from sklearn. preprocessing import OneHotEncoder
from sklearn. linear_model import LogisticRegression
from sklearn import metrics
from sklearn.metrics import log_loss, roc_auc_score


In [24]:
def feature_engineering(dataFrame: pd.DataFrame):
    # отделяем последний день
    splitDate = datetime.date(2021, 10, 2)
    train = dataFrame[dataFrame['date'] != splitDate]
    lastDay = dataFrame[dataFrame['date'] == splitDate]
    # делим на тренировочную и тестовую (для баннеров 0 и 1 отдельно) выборки и применяем onehotEncoding
    X_train = train[['hour', 'zone_id', 'banner_id', 'os_id', 'oaid_hash']].values
    y_train = train['clicks']
    X_test_0 = lastDay[['hour', 'zone_id', 'banner_id0', 'os_id', 'oaid_hash']].values
    X_test_1 = lastDay[['hour', 'zone_id', 'banner_id1', 'os_id', 'oaid_hash']].values
    ips_test = lastDay[['g0', 'g1', 'coeff_sum0', 'coeff_sum1']]
    y_test = lastDay['clicks']
    
    encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=True)
    X_train = encoder.fit_transform(X_train)
    X_test_0 = encoder.transform(X_test_0)
    X_test_1 = encoder.transform(X_test_1)
    return X_train, X_test_0, X_test_1, y_train, y_test, ips_test

X_train, X_test_0, X_test_1, y_train, y_test, ips_test = feature_engineering(data)
print(X_train, X_test_0, X_test_1, y_train, y_test, ips_test)

  (0, 0)	1.0
  (0, 24)	1.0
  (0, 3348)	1.0
  (0, 4920)	1.0
  (0, 3475784)	1.0
  (1, 22)	1.0
  (1, 25)	1.0
  (1, 3349)	1.0
  (1, 4920)	1.0
  (1, 3183237)	1.0
  (2, 23)	1.0
  (2, 26)	1.0
  (2, 3350)	1.0
  (2, 4920)	1.0
  (2, 1362761)	1.0
  (3, 0)	1.0
  (3, 27)	1.0
  (3, 3351)	1.0
  (3, 4921)	1.0
  (3, 3843497)	1.0
  (4, 0)	1.0
  (4, 28)	1.0
  (4, 3352)	1.0
  (4, 4921)	1.0
  (4, 2932782)	1.0
  :	:
  (13677644, 13)	1.0
  (13677644, 43)	1.0
  (13677644, 3505)	1.0
  (13677644, 4920)	1.0
  (13677644, 5476975)	1.0
  (13677645, 20)	1.0
  (13677645, 38)	1.0
  (13677645, 3600)	1.0
  (13677645, 4920)	1.0
  (13677645, 198273)	1.0
  (13677646, 22)	1.0
  (13677646, 36)	1.0
  (13677646, 3370)	1.0
  (13677646, 4921)	1.0
  (13677646, 283630)	1.0
  (13677647, 0)	1.0
  (13677647, 985)	1.0
  (13677647, 3369)	1.0
  (13677647, 4920)	1.0
  (13677647, 4276831)	1.0
  (13677648, 7)	1.0
  (13677648, 43)	1.0
  (13677648, 3981)	1.0
  (13677648, 4920)	1.0
  (13677648, 5372593)	1.0   (0, 0)	1.0
  (0, 32)	1.0
  (0, 34

In [15]:

# создаём модель
def create_model():
    return LogisticRegression(solver="liblinear", penalty="l2", C=0.001)

model = create_model()
model.fit(X_train, y_train)

LogisticRegression(C=0.001, solver='liblinear')

In [18]:
    
baselineMean = y_train.mean()
baselineLogLoss = log_loss(y_test, [baselineMean] * len(y_test))
baselineAuc = roc_auc_score(y_test, [baselineMean] * len(y_test))

print('Вывод для бейзлайнa')
print('log_loss: ', baselineLogLoss)
print('auc: ', baselineAuc)

y_pred = model.predict_proba(X_test_1)
modelLogLoss = log_loss(y_test, y_pred)
modelAuc = roc_auc_score(y_test, y_pred[:, 1])

print('Вывод для модели')
print('log_loss: ', modelLogLoss)
print('auc: ', modelAuc)

Вывод для бейзлайнa
log_loss:  0.15428600773285636
auc:  0.5
Вывод для модели
log_loss:  0.13877345294543336
auc:  0.7578691825449726


In [ ]:
# Преобразуем вероятность того, что одна случайная величина болтше другой:
# P(X > Y) = P(X - Y > 0) = 1 - P(X - Y <= 0) = 1 - F_{X - Y}(0)
# Причём, величина X - Y является нормальной. Тогда надо посчитать разность coeff_sum0 и coeff_sum1 и корень из суммы квадратов g0 и g1

In [22]:
from scipy.stats import norm
from scipy.special import logit

In [34]:
lambda_=10
pi_0 =  1 - norm.cdf(0, loc=ips_test['coeff_sum0'] - ips_test['coeff_sum1'], scale=np.sqrt(ips_test['g0']**2 + ips_test['g1']**2))
coeff_sum0 = logit(model.predict_proba(X_test_0)[:, 1])
coeff_sum1 = logit(model.predict_proba(X_test_1)[:, 1])
pi_1 =  1 - norm.cdf(0, loc=coeff_sum0 - coeff_sum1, scale=np.sqrt(ips_test['g0']**2 + ips_test['g1']**2 + 1e-9))
cips = np.mean(y_test * np.minimum(pi_1 / (pi_0 + 1e-9), lambda_))
print('Clipped IPS')
print(cips)

Clipped IPS
0.06804282816928688
